## DALEX

DALEX (независимый от модели язык для исследования и объяснения) является одним из немногих широко используемых фреймворков XAI, который пытается охватить большинство аспектов объяснимости. DALEX не зависит от модели и может предоставить некоторые метаданные о базовом наборе данных, чтобы придать некоторый контекст объяснению. Этот фреймворк дает вам представление о производительности модели и справедливости модели, а также обеспечивает объяснимость глобальной и локальной моделей.

In [1]:
!pip install --upgrade pandas numpy matplotlib seaborn scikit-learn dalex

  Using cached numpy-1.23.2-cp39-cp39-win_amd64.whl (14.7 MB)


In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

import dalex as dx 

import matplotlib.pyplot as plt
import seaborn as sns

from IPython import display

np.random.seed(123)

О данных

В этом уроке мы будем использовать набор данных для прогнозирования положения клуба FIFA, используемый в учебном пособии Explainerdashboard и в учебном пособии H2O AutoML Expainer. Но вместо того, чтобы использовать этот набор данных для прогнозирования положения конкретного клуба в лиге на основе способностей его игрока, мы попытаемся предсказать оценку игрока на основе его навыков и способностей. Итак, это будет проблема регрессии, и мы будем использовать DALEX для объяснения регрессионной модели ML.

In [ ]:
# We will read the training data
df_train = pd.read_csv('C:\\Users\\1\\Downloads\\fifa_league_21_train.csv')

In [ ]:
df_train.head()

In [ ]:
# Reading the dataset description
with open('datasets/FIFA_Club_Position/data_description.txt') as f:
    contents = f.read()
    print(contents)

In [3]:
# Check dataset dimensions
print(df_train.shape)

NameError: name 'df_train' is not defined

In [ ]:
df_train.columns

In [ ]:
df_train.info()

Как мы можем видеть, набор данных содержит около 81 характеристики, из которых входные значения мы сохраним только информацию о навыках и способностях игрока на разных игровых позициях, а также возраст игрока для прогнозирования оценки игрока. Возраст будет интересной особенностью, так как обычно молодые игроки имеют гораздо более высокую оценку из-за их растущего потенциала. Мы отбросим другие функции для этого конкретного анализа. Итак, давайте начнем!

In [ ]:
features_to_drop = ['short_name', 'club_name', 'league_name', 'preferred_foot', 'wage_eur', 'club_position', 'position']
target_variable = 'value_eur'

In [ ]:
df_train.drop(columns=features_to_drop, inplace=True)

In [ ]:
print(df_train.shape)

In [ ]:
# Let's check missing values

sns.displot(
    data=df_train.isna().melt(value_name="missing"),
    y="variable",
    hue="missing",
    multiple="fill",
    height=10,
    aspect=1.2
)
plt.show()

Анализируя пропущенные значения, мы видим, что скорость вратарей имеет высокий процент пропущенных значений. Следовательно, мы также можем отказаться от этой функции. Но для других недостающих значений, связанных с темпом, стрельбой, передачей, дриблингом, защитой и физической подготовкой, в основном эти данные недоступны для вратарей. Итак, вместо того, чтобы отбрасывать записи, связанные с вратарями, мы сохраним постоянное значение 40 для всех вратарей, поскольку интуитивно думаем, что, поскольку эти вратари являются профессиональными игроками, почти у всех из них будет оценка не менее 40. В разделе value_eur отсутствует одно значение. Весьма возможно, что игрок решил уйти в отставку, и, следовательно, значение не присваивается. Итак, мы просто отбросим эту запись как выброшенную.

In [ ]:
# dropping the feature 'goalkeeping_speed'
df_train.drop(columns=['goalkeeping_speed'], inplace=True)

In [ ]:
# data imputation for the features 'pace', 'shooting', 'passing', 'dribbling', 'defending' and 'physic' with a constant value of 40
for features in ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']:
    df_train[features].fillna(40, inplace=True)

In [ ]:
# Consider all other missing values as outliers, so just drop them
df_train.dropna(inplace=True)

In [ ]:
# Let's separate the labels from training data
labels = df_train[target_variable]
df_train.drop(columns=[target_variable], inplace=True)

In [ ]:
print(df_train.shape)

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

Training the model

In [ ]:
x_train,x_valid,y_train,y_valid = train_test_split(df_train,labels,test_size=0.2,random_state=123)

In [ ]:
model = RandomForestRegressor(n_estimators=790, min_samples_split = 3, random_state=123).fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_valid)
print('R^2 score is:')
r2_score(y_valid, y_pred)

In [ ]:
model.score(x_valid, y_valid) # 790 best so far, mss = 3 - 0.9364

Model evaluation on unseen data

In [ ]:
df_test = pd.read_csv('datasets/fifa_league_22_test.csv')

def predict_player_valuation(df_test, model, player_name):
    '''
    Predict club position from player quality
    '''
    
    df_test = df_test[df_test['short_name'] == player_name]
    display.display(df_test)
    print(f'Original Valuation for the 2022-2023 season for {player_name} is EUR. {df_test[target_variable].values[0]}')
    df_test.drop(columns=['short_name', 
                          'club_name', 
                          'league_name', 
                          'preferred_foot', 
                          'wage_eur', 
                          'club_position'], inplace=True)
    df_test.drop(columns=['goalkeeping_speed'], inplace=True)
    df_test.drop(columns=[target_variable], inplace=True)
    for features in ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']:
            df_test[features].fillna(40, inplace=True)
    
    pred = model.predict(df_test)
    print(f'Predicted Valuation for the 2022-2023 season for {player_name} is EUR. {round(pred[0], 1)}')

Теперь давайте применим модель к молодому и многообещающему игроку Джейдону Санчо, который перешел в "Манчестер Юнайтед" из дортмундской "Боруссии" на сезон 2022-2023 годов за ошеломляющие 85 миллионов евро

In [ ]:
predict_player_valuation(df_test, model, player_name = 'J. Sancho')

Интересно, что модель была очень близка! Первоначальная оценка Jadon Sancho в евро составляла около 116 млн евро, но модель предсказала, что она составит 112,25 млн евро, что довольно близко! Теперь давайте попробуем DALEX для объяснения модели и расшифруем, почему предсказание модели было близким!

Модельный агностический язык для исследования и объяснения (DALEX) - это независимый от модели метод надежной объяснимости, используемый как для глобальных, так и для локальных объяснений. Взгляните на этот учебник по DALEX https://dalex.drwhy.ai/python-dalex-fifa.html для справки. Как указано в исходном репозитории проекта DALEX (https://github.com/ModelOriented/DALEX ), DALEX может решать различные аспекты объяснимости -

Равномерное отклонение от моделей прогнозирования,
надежная структура API (согласованная грамматика для анализа модели),
локальное объяснение (объяснение на уровне прогнозирования),
глобальное объяснение (объяснение на уровне модели),
проверка предвзятости и справедливости,
интерактивная визуализация (ARENA).

Теперь давайте начнем с создания нашего объяснителя модели DALEX.

In [ ]:
# Create DALEX Explainer object 
explainer = dx.Explainer(model, 
                         x_valid, y_valid, 
                         model_type = 'regression',
                         label='Random Forest')

Объект explainer предоставляет некоторые метаданные о обученной модели, чтобы задать контекст объяснения.

Давайте начнем с объяснений на уровне модели. Получение предварительных знаний о производительности модели помогает установить правильные ожидания и, следовательно, может быть важным шагом для объяснения модели.

In [ ]:
model_performance = explainer.model_performance("regression")
display.display(model_performance.result)

Далее мы будем использовать DALEX, чтобы получить глобальное объяснение, визуализируя важность функции.

In [ ]:
# As shown in https://dalex.drwhy.ai/python-dalex-fifa.html, we will group the features for understanding the key feature contributions in specific areas
variable_groups = {
    'international_reputation' : ['international_reputation'],
    'age': ['age'],
    'attacking': ['attacking_crossing',
       'attacking_finishing', 'attacking_heading_accuracy',
       'attacking_short_passing', 'attacking_volleys'],
    'skill': ['skill_dribbling', 'skill_moves',
       'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
       'skill_ball_control'],
    'movement': ['movement_acceleration', 'movement_sprint_speed',
       'movement_agility', 'movement_reactions', 'movement_balance'],
    'power': ['power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
       'power_long_shots'],
    'mentality': ['mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure'],
    'defending': ['defending_marking_awareness', 'defending_standing_tackle',
       'defending_sliding_tackle'],
    'goalkeeping' : ['goalkeeping_diving',
       'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning', 'goalkeeping_reflexes'],
    'abilities': ['overall', 'potential', 'pace', 'shooting', 
                  'passing','dribbling','defending','physic', 'weak_foot'],
}

In [ ]:
Var_Importance = explainer.model_parts(variable_groups=variable_groups, B=15, random_state=123)
Var_Importance.plot(max_vars=10, 
                    rounding_function=np.rint, 
                    digits=None, 
                    vertical_spacing=0.15,
                    title = 'Feature Importance' )

Как мы можем видеть, характеристика способностей является наиболее важной характеристикой в соответствии с моделью прогнозирования оценки игрока. Аналогичным образом мы можем видеть важность других объектов в наборе данных в порядке от более высокой важности к более низкой важности.

Далее мы будем использовать графики частичной зависимости, описанные в главе 2, для глобальных объяснений.

In [ ]:
pdp = explainer.model_profile(type = 'partial', N=800)

pdp.plot(variables = ['age', 'potential'])

Другой глобальный подход к объяснению агрегированного эффекта каждого признака может быть получен с помощью накопленного профиля локальной зависимости.

In [ ]:
ald = explainer.model_profile(type = 'accumulated', N=800)

ald.plot(variables = ['age', 'movement_reactions'])

Мы видели лишь несколько примеров. Пожалуйста, не стесняйтесь исследовать больше.

Далее мы будем использовать DALEX для локальных объяснений или объяснений на уровне прогнозирования. Мы будем использовать два подхода, предоставляемые DALEX - Interactive Breakdown и SHAP.

In [ ]:
prediction_level = {'interactive_breakdown':[], 'shap':[]}

for name in ['Cristiano Ronaldo', 'L. Messi', 'J. Sancho']:
    player = df_test[df_test['short_name'] == name].copy()
    player.drop(columns=['short_name', 
                          'club_name', 
                          'league_name', 
                          'preferred_foot', 
                          'wage_eur', 
                          'club_position'], inplace=True)
    player.drop(columns=['goalkeeping_speed'], inplace=True)
    player.drop(columns=[target_variable], inplace=True)
    for features in ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']:
            player[features].fillna(40, inplace=True)
            
    ibd = explainer.predict_parts(player, type='break_down_interactions', label=name)
    sh = explainer.predict_parts(player, type='shap', B=10, label=name)
    
    prediction_level['interactive_breakdown'].append(ibd)
    prediction_level['shap'].append(sh)

Разбейте взаимодействия для локального объяснения прогнозируемой оценки для трех лучших футболистов мира - Криштиану Роналду, Лионеля Месси и Джейдона Санчо.

In [ ]:
prediction_level['interactive_breakdown'][0].plot(prediction_level['interactive_breakdown'][1:3],
                  rounding_function=lambda x, 
                  digits: np.rint(x, digits).astype(np.int),
                  digits=None, 
                  max_vars=15)

Функция SHAP важна для локального объяснения прогнозируемой оценки трех лучших футболистов мира - Криштиану Роналду, Лионеля Месси и Джейдона Санчо.

In [ ]:
prediction_level['shap'][0].plot(prediction_level['shap'][1:3],
                 rounding_function=lambda x, 
                 digits: np.rint(x, digits).astype(np.int),
                 digits=None, 
                 max_vars=15)

Было очень интересно узнать, что как для Криштиану Роналду, так и для Лионеля Месси, хотя они обладают высочайшими способностями и, бесспорно, являются лучшими футболистами в мире, DALEX смог определить, что оба они стареют, и, следовательно, возраст персонажа негативно влияет на прогноз модели, тем самым снижая их прогнозируемый уровень. ценность оценки. Напротив, поскольку Джейдон Санчо - молодой талант, возраст влияет положительно и пытается увеличить прогнозируемую оценку игрока. Это абсолютно логично, и, следовательно, предоставленное местное объяснение очень уместно!

Далее давайте рассмотрим профили при прочих равных условиях с использованием DALEX. При прочих равных условиях профили помогают нам выполнять анализ "Что, если" и показывают, как изменится прогноз модели, если изменится значение отдельного признака, что больше похоже на анализ чувствительности, описанный в главе 2.

In [ ]:
ceteris_paribus_profile = explainer.predict_profile(player,
                               variables=['age', 'potential'],
                               label=name) # variables to calculate 

ceteris_paribus_profile.plot(size=3, title= f"What If? {name}") # larger width of the line and dot size & change title

Как мы можем видеть из приведенных выше графиков, для Джейдона Санчо по мере его старения модель предсказывает падение его оценки. Но если его потенциальные способности увеличиваются, его оценка также может увеличиться (как показано на правой диаграмме).

Проверка справедливости модели - еще одна важная функция DALEX. Давайте посмотрим, свободна ли наша модель от каких-либо предубеждений и справедлива ли она! Для получения более подробной информации о проверке справедливости модели с помощью DALEX, пожалуйста, обратитесь к этому - https://dalex.drwhy.ai/python-dalex-fairness.html .

Давайте создадим защищенную переменную и привилегированную переменную для проверки справедливости. Мы проведем эту проверку справедливости для трех разных групп игроков в зависимости от их возраста. Все игроки младше 20 лет считались "молодежными" игроками, игроки в возрасте от 20 до 30 лет считались развивающимися игроками, а игроки старше 30 лет считались "развитыми" игроками. Теперь давайте проверим нашу честность с помощью DALEX.

In [ ]:
protected = np.where(x_valid.age < 30, np.where(x_valid.age < 20, 'youth', 'developing'), 'developed')

privileged = 'youth'

In [ ]:
fairness = explainer.model_fairness(protected = protected, privileged=privileged)
fairness.fairness_check(epsilon = 0.7)

In [ ]:
fairness.result

In [ ]:
fairness.plot(type = 'density')

К счастью, смещения не обнаружено, и наша модель кажется справедливой, учитывая нашу привилегированную переменную. Для этой проблемы, возможно, предвзятость и справедливость не так критичны, но DALEX действительно является очень хорошей основой для рассмотрения предвзятости и справедливости в моделях.

Степень объяснимости часто возрастает по мере взаимодействия с пользователем. У ДАЛЕКСОВ есть такая возможность использовать АРЕНУ - https://dalex.drwhy.ai/python-dalex-arena.html , что позволяет нам создавать интерактивную панель мониторинга с различными аспектами объяснимости модели с помощью DALEX.

In [ ]:
arena_dataset = df_test[:400].set_index('short_name')

In [ ]:
# create empty Arena
arena = dx.Arena()
# push created explainer
arena.push_model(explainer)
# push whole test dataset (including target column)
arena.push_observations(arena_dataset)
# run server on port 9294
arena.run_server(port=9294)

In [ ]:
arena.print_options()

In [ ]:
arena.stop_server()

In [ ]:
Перейдите к https://arena .drwhy.ai/?data=http://127.0.0.1:9294 / для панели управления Ареной. Вы можете использовать json Arena config, используемый для этого руководства, из https://raw.githubusercontent.com/PacktPublishing/Applied-Machine-Learning-Explainability-Techniques/main/Chapter09/dalex_sessions/session-1647894542387.json. Не стесняйтесь создавать и проектировать интерактивную панель мониторинга с помощью Arena от DALEX.

DALEX очень надежен! В нем рассматриваются различные аспекты объяснимости, от набора данных до моделей, от локальной объяснимости до глобальной объяснимости, и он не зависит от модели! Есть еще много примеров, доступных по адресу - https://github.com/ModelOriented/DALEX , пожалуйста, изучите их все. Но DALEX, похоже, ограничен структурированными данными. Я думаю, что в будущем, если сделать DALEX легко применимым к изображениям и текстовым данным, это увеличит его распространение в сообществе исследователей искусственного интеллекта.